In [2]:
# This is a new library compared to the previous modules. 
# Please perform "pip install sentence_transformers==2.7.0"
from sentence_transformers import SentenceTransformer

# if you get an error do the following:
# 1. Uninstall numpy 
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch
# run the above cell, it should work
model = SentenceTransformer("multi-qa-distilbert-cos-v1")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/523 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
user_question = "I just discovered the course. Can I still join it?"

In [4]:
em = model.encode(user_question)

In [6]:
em[0]


0.078222655

In [7]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [12]:
course = 'machine-learning-zoomcamp'
filtered_data = [item for item in documents if item['course'] == course]

In [13]:
len(filtered_data)

375

In [14]:
import numpy as np

In [15]:
from tqdm.auto import tqdm

In [16]:
embeddings = []
for doc in tqdm(filtered_data):
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'

    qa_text_vector = model.encode(qa_text)
    embeddings.append(qa_text_vector)
    
X = np.array(embeddings)
X.shape
    

  0%|          | 0/375 [00:00<?, ?it/s]

(375, 768)

In [17]:
scores = X.dot(em)

In [19]:
scores

array([ 2.89217263e-01,  4.35050726e-01,  2.20572561e-01,  1.28085926e-01,
        3.28754455e-01,  4.58999664e-01,  4.48930591e-01,  2.70170599e-01,
        2.13975310e-01,  3.39745760e-01,  1.63339168e-01,  3.15918088e-01,
        1.53621323e-02,  4.14240420e-01,  6.50657296e-01,  4.12786484e-01,
        2.02390462e-01,  4.71707582e-01,  3.74821007e-01,  3.53319108e-01,
        4.50558245e-01,  1.85430944e-01,  3.45526993e-01,  8.22948217e-02,
       -2.19077598e-02, -4.64250743e-02,  8.60690773e-02,  9.87197682e-02,
       -2.14188285e-02,  1.28751006e-02,  2.54870541e-02,  3.66866291e-02,
        4.56338078e-02, -2.05279887e-02,  1.82662532e-02,  3.39789167e-02,
        1.66510604e-03,  1.03209512e-02,  3.40170749e-02, -2.65231431e-02,
       -4.11909968e-02, -4.97983992e-02, -6.20346665e-02, -7.63561390e-03,
        4.61616889e-02,  6.95944112e-03,  4.77613509e-02, -9.97766852e-04,
       -5.60126789e-02, -5.96206151e-02,  2.05209196e-01, -1.16968602e-01,
       -2.34160442e-02, -

In [20]:
np.max(scores)

0.6506573

In [21]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_data, embeddings=X)
search_engine.search(em, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [22]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [24]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [25]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    question_emb = model.encode(q['question'])
    results = search_engine.search(question_emb, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [26]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [27]:
hit_rate(relevance_total)

0.9398907103825137

======= Elastic search

In [28]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

In [31]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [32]:
from tqdm.auto import tqdm

In [33]:
for i, doc in enumerate(tqdm(filtered_data)):
    question = doc['question']
    text = doc['text']
    
    # Adding the question_text_vector to the document
    filtered_data[i]['question_text_vector'] = X[i]

  0%|          | 0/375 [00:00<?, ?it/s]

In [34]:
for doc in tqdm(filtered_data):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/375 [00:00<?, ?it/s]

In [48]:
def elastic_search_knn(vector):
    knn = {
        "field": 'question_text_vector',
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )

    # print(es_results['hits']['max_score'])
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [46]:
elastic_search_knn(em)

0.82532895


[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the vid

In [49]:
relevance_total_es = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    question_emb = model.encode(q['question'])
    results = elastic_search_knn(question_emb)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total_es.append(relevance)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [50]:
hit_rate(relevance_total_es)

0.9398907103825137